In [1]:
import pickle
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from alexnet import AlexNet

nb_classes = 43
epochs = 10
batch_size = 128

with open('./train.p', 'rb') as f:
    data = pickle.load(f)

X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], test_size=0.33, random_state=0)
tf.compat.v1.disable_eager_execution()
features = tf.compat.v1.placeholder(tf.float32, (None, 32, 32, 3))
labels = tf.compat.v1.placeholder(tf.int64, None)
resized = tf.compat.v1.image.resize_images(features, (227, 227))

# Returns the second final layer of the AlexNet model,
# this allows us to redo the last layer for the traffic signs
# model.
fc7 = AlexNet(resized, feature_extract=True)
fc7 = tf.stop_gradient(fc7)
shape = (fc7.get_shape().as_list()[-1], nb_classes)
fc8W = tf.Variable(tf.compat.v1.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.matmul(fc7, fc8W) + fc8b

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
loss_op = tf.reduce_mean(cross_entropy)
opt = tf.compat.v1.train.AdamOptimizer()
train_op = opt.minimize(loss_op, var_list=[fc8W, fc8b])
init_op = tf.compat.v1.global_variables_initializer()

preds = tf.argmax(logits, 1)
accuracy_op = tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))


def eval_on_data(X, y, sess):
    total_acc = 0
    total_loss = 0
    for offset in range(0, X.shape[0], batch_size):
        end = offset + batch_size
        X_batch = X[offset:end]
        y_batch = y[offset:end]

        loss, acc = sess.run([loss_op, accuracy_op], feed_dict={features: X_batch, labels: y_batch})
        total_loss += (loss * X_batch.shape[0])
        total_acc += (acc * X_batch.shape[0])

    return total_loss/X.shape[0], total_acc/X.shape[0]

saver = tf.compat.v1.train.Saver()
save_file = "model_data/LeNet_For_Traffic_Sign.ckpt"

with tf.compat.v1.Session() as sess:
    sess.run(init_op)

    for i in range(epochs):
        # training
        X_train, y_train = shuffle(X_train, y_train)
        t0 = time.time()
        for offset in range(0, X_train.shape[0], batch_size):
            end = offset + batch_size
            sess.run(train_op, feed_dict={features: X_train[offset:end], labels: y_train[offset:end]})

        val_loss, val_acc = eval_on_data(X_val, y_val, sess)
        print("Epoch", i+1)
        print("Time: %.3f seconds" % (time.time() - t0))
        print("Validation Loss =", val_loss)
        print("Validation Accuracy =", val_acc)
        print("")
        
    saver.save(sess, save_file)
    print("Model saved")

Epoch 1
Time: 260.697 seconds
Validation Loss = 0.5111002340937003
Validation Accuracy = 0.8708555529747581

Epoch 2
Time: 254.223 seconds
Validation Loss = 0.33921934867054465
Validation Accuracy = 0.9147538449878679

Epoch 3
Time: 252.134 seconds
Validation Loss = 0.2718504589338632
Validation Accuracy = 0.9292062756239294

Epoch 4
Time: 249.217 seconds
Validation Loss = 0.21893784625608187
Validation Accuracy = 0.9483731354772699

Epoch 5
Time: 249.458 seconds
Validation Loss = 0.19096368827879895
Validation Accuracy = 0.950846278687719

Epoch 6
Time: 250.254 seconds
Validation Loss = 0.1729978529933763
Validation Accuracy = 0.9555607079326374

Epoch 7
Time: 250.585 seconds
Validation Loss = 0.15835430893267644
Validation Accuracy = 0.9591931370506239

Epoch 8
Time: 250.179 seconds
Validation Loss = 0.14077563222233686
Validation Accuracy = 0.9643712806198621

Epoch 9
Time: 249.947 seconds
Validation Loss = 0.13309085925125302
Validation Accuracy = 0.9649895664224744

Epoch 10
Time: